This Notebook evaluates a convergence study for the 1D Shock-Acoustic-Wave interaction problems as presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [ ]:
#r "BoSSSpad.dll"
#r "XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [ ]:
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D_ConvergenceStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


In [ ]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


In [ ]:
var sessions = database.Sessions;
sessions

In [ ]:
var aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

## Choose which runs to plot

In [ ]:
var sessions=database.Sessions.Where(s => (s.Name.Contains("sP1.5") &&s.Name.Contains("ampneg0")&&s.Name.Contains("amppos1")&& !s.Name.Contains("241"))); // the one plotted in the publication
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions


# A Helper Function to obtain different plot formats

In [ ]:
using BoSSS.Solution.Tecplot;
using System.IO;
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Residual History

In [ ]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

helper function to evaluate the XDG field along a x-ray for a specific time

In [ ]:
(double[] tVals, double[]yVals) GetPlotAlongXRay(double tMin,double tMax, double x, int nRef,string field,ITimestepInfo ti){
//double shockPosition=0.5;
var p1= new double[] {x,tMin};
var p2=new double[] {x,tMax};

//timesteps to compare

// # use p_per from BoSSS 
var pEnd =ti.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] tVals= GenericBlas.Linspace(tMin,tMax,nRef);
return(tVals,yVals);
}


helper function to evaluate the XDG field along a t-ray for a point x

In [ ]:
(double[] xVals, double[]yVals) GetPlotAlongTRay(double xMin,double xMax, double t, int nRef,string field,ITimestepInfo ti){
//double shockPosition=0.5;
var p1= new double[] {xMin,t};
var p2=new double[] {xMax,t};

//timesteps to compare

// # use p_per from BoSSS 
var pEnd =ti.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

helper function to compute the error metrics (i.e. the deviation of each amplitude from the exact amplification/reduction factor)

In [ ]:
List<double> ComputeAmpErrors(double tMin,double tMax, double xMin, double xMax, int xRef, int tRef,string field,ITimestepInfo ti, bool isMax,double exact_amp_fac){
    if(true){
        var xVals= GenericBlas.Linspace(xMin,xMax,xRef);
        var maxminVals = new List<double>();
        foreach(var x in xVals ){
            (double[] tVals, double[] yVals)=GetPlotAlongXRay(tMin,tMax,x,tRef,field,ti);
            if(isMax){
                maxminVals.Add(Math.Abs((exact_amp_fac-yVals.Max())/exact_amp_fac));
            }else{
                maxminVals.Add(Math.Abs((exact_amp_fac-yVals.Min())/exact_amp_fac));
            }
        }
        return maxminVals;
    }else{
        var pEnd =ti.GetField(field);
        var fullcm= CellMask.GetFullMask(pEnd.GridDat);
        var ba= new bool[fullcm.ItemEnum.Count()];
        
        var bb = new BoSSS.Platform.Utils.Geom.BoundingBox(new double[,] {{0,0},{1,1}});
        foreach(var cell in fullcm.ItemEnum){
            pEnd.GridDat.iGeomCells.GetCellBoundingBox(cell, bb);
            var min = bb.Min;
            if(!isMax){
                if( min[0]>=xMin && min[0]<=xMax){
                    ba[cell]=true;
                };
            }else{
                if( min[0]>=xMin && min[0]<=xMax ){
                    ba[cell]=true;
                };
            }

        }
        var cm =new CellMask(pEnd.GridDat,new BitArray(ba));
        var valMin=new double[cm.ItemEnum.Count()];
        var valMax=new double[cm.ItemEnum.Count()];
        pEnd.GetCellwiseExtremalValues(valMin, valMax,cm);
        List<double> maxminVals;
        if(isMax){
            maxminVals=new List<double> {Math.Abs((exact_amp_fac-valMax.Max())/exact_amp_fac)};
        }else{
            maxminVals=new List<double> {Math.Abs((exact_amp_fac-valMin.Min())/exact_amp_fac)};
        }
        return maxminVals;
    }
    
}

var eps=0.000001;
double xMin=1.8 +eps;double xMax=2.0-eps; int xRef=5; 
double tMin =0 + eps;int tRef=800;
GenericBlas.Linspace(xMin,xMax,xRef)

In [ ]:
var ses= sessions;
ses

choose the sessions

they must be reordered for the plot to work...

In [ ]:
var ses= sessions;
ses = ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses

Number of cells

In [ ]:
int[] Cells = ses.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

helper arrays

In [ ]:
int[] degs = new int[] {0,1,2,3};
//degs = new int[] {0};
var errors_array= MultidimensionalArray.Create(degs.Count(),ses.Count());


Function used to calculate the EOCs

In [ ]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

Function used to compute the ideal convergence plot

In [ ]:
List<double> GetIdealConvPlot(List<double> errors,List<double> cellSize, int p){
        var idealErrors = new List<double>();

    // Calculate ideal errors for each cell size
    for (int i = 0; i < cellSize.Count; i++)
    {
        double idealError = errors[0] * Math.Pow(cellSize[i] / cellSize[0], p + 1);
        idealErrors.Add(idealError);
    }

    return idealErrors;
} 

compute the exact analytical pressure amplification/reduction factor for both cases investigated

In [ ]:
// Given variables
var MBaseL = 1.5; // Base Mach number on the left
var gamma = 1.4; //Specific heat ratio

// Calculate right Mach number (Mach_R)
var MBaseR  = Math.Sqrt((1 + ((gamma-1)/2) * MBaseL*MBaseL) / (gamma * MBaseL*MBaseL - (gamma-1)/2));

// Upstream fast acoustic wave amplification coefficient
var delta_pPrR_over_delta_pPrL = ((1+MBaseL)*(1+MBaseL)) / (1+2*MBaseR+1/MBaseL/MBaseL) * (1 - ((gamma-1)/(gamma+1)) * (1-(1/MBaseL))* (1-(1/MBaseL)));

// Downstream slow acoustic wave amplification coefficient
var delta_pPrR_over_delta_pPrMiR = -(1- 2*MBaseR +1 / MBaseL/MBaseL) / (1+ 2*MBaseR +1 / MBaseL/ MBaseL);
(delta_pPrR_over_delta_pPrL,delta_pPrR_over_delta_pPrMiR)

Create the convergence plot

In [ ]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
int count=0;
MultidimensionalArray errorsAndCells=MultidimensionalArray.Create(ses.Count(),degs.Length +1);
MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(ses.Count(),degs.Length+1);
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =0;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        Console.Write($"P={p},sess={iSess}");
        //extract the relevant field and timestep Index
        var timesteps = si.Timesteps;
        var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
        if(rTSIndex !=-1){
            IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
            var fields = lts.Fields.ToList();
            cells.Add(Math.Sqrt(fields[0].Basis.GridDat.iGeomCells.GetCellVolume(0)));
            //extract AmpError
            // if(si.Name.Contains("sP0.5")){
            //     double exact_amp_fac=delta_pPrR_over_delta_pPrMiR*1e-05; double tMin=0.0 +eps;double tMax=4.0-eps;
            //     List<double> ampErrors= ComputeAmpErrors(tMin,tMax, xMin, xMax, xRef,tRef,"p_per",lts,false,exact_amp_fac);
            //     //errors_array[p,iSess]=ampErrors.Average();
            //     errors.Add(ampErrors.Average());
            //     Console.WriteLine($" sp0.5, SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}");
            // }else{
            //     double exact_amp_fac=delta_pPrR_over_delta_pPrL*1e-05; double tMin=0.0 +eps;double tMax=1.0-eps;
            //     List<double> ampErrors= ComputeAmpErrors(tMin,tMax, xMin, xMax, xRef,tRef,"p_per",lts,true,exact_amp_fac);
            //     //errors_array[p,iSess]=ampErrors.Average();
            //     errors.Add(ampErrors.Average());
            //     Console.WriteLine($" sp1.5, SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}");
            // }
             if(si.Name.Contains("sP0.5")){

                double t=2.5;double xMin=0.6;double xMax=2.999; int xRef=800;
                (double[] xVals_dummy, double[] yVals) = GetPlotAlongTRay(xMin,xMax,t,xRef,"p_per",lts);
                var ex_amp=delta_pPrR_over_delta_pPrMiR*1e-5;
                errors.Add(Math.Abs(yVals.Min()-ex_amp)/ex_amp);
                Console.WriteLine($" sp0.5, SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}, amp:{yVals.Min()}");
                
            }else{

                double t=0.8;double xMin=1.6;double xMax=2.999; int xRef=800;
                (double[] xVals_dummy, double[] yVals) = GetPlotAlongTRay(xMin,xMax,t,xRef,"p_per",lts);
                var ex_amp=delta_pPrR_over_delta_pPrL*1e-5;
                errors.Add(Math.Abs(yVals.Max()- ex_amp)/ ex_amp);
                Console.WriteLine($" sp1.5, SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}, amp:{yVals.Max()}");
                
            }
            
        }
    }
    errorsAndCells.SetColumn(0,cells);
    errorsAndCellsIdeal.SetColumn(0,cells);
    errorsAndCells.SetColumn(p+1,errors);
    errorsAndCellsIdeal.SetColumn(p+1,GetIdealConvPlot(errors,cells,p));
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    var fmt = GetFormat(count);
    fmt.PointSize=0.01;
    fmt.DashType=DashTypes.Dashed;

    plot.AddDataGroup($"ideal {p+1}",cells,GetIdealConvPlot(errors,cells,p), fmt);
    count++;
}
errorsAndCells.SaveToTextFile("ConvergenceStudyErrors.txt");
errorsAndCellsIdeal.SaveToTextFile("ConvergenceStudyErrorsIdeal.txt");
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:500)

In [ ]:
EOCs

Assertion

In [ ]:
NUnit.Framework.Assert.IsTrue(EOCs[0]>0.01);

In [ ]:
NUnit.Framework.Assert.IsTrue(EOCs[1]>1.2);

In [ ]:

NUnit.Framework.Assert.IsTrue(EOCs[2]>2.9);

In [ ]:
NUnit.Framework.Assert.IsTrue(EOCs[3]>3.9);

plot it

In [ ]:
// using BoSSS.Solution.Tecplot;
// using System.IO;
// Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
// IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
// //double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
// var allTs = (List<double>) lts.TimeStepNumbers; 
// var N=40;
// var TStoPlot= allTs.Where(ts => ts % N == 0); // plot every N
// foreach(var si in ses){
//     foreach(var timestep in si.Timesteps){
//         if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
//                 IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
//                 var SF = lts.GetShadowFields();
//                 var texplot = new Tecplot(timestep.Fields.Pick(0).GridDat, 2);
//                 texplot.PlotFields( si.Name + "_" + lts.TimeStepNumber,0.0,timestep.Fields);
//         }
//     }
// }